In [1]:
import bw2data
from edges import EdgeLCIA, get_available_methods

In [2]:
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
#bw2data.projects.set_current("ecoinvent-3.10-cutoff")
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
#act = [a for a in bw2data.Database("ecoinvent-3.10-cutoff") if "NMC" in #a["name"]][0]
act

'electricity production, natural gas, combined cycle power plant' (kilowatt hour, CN-HL, None)

In [3]:
[m for m in get_available_methods() if "ImpactWorld" in str(m)]

[('ImpactWorld+ 2.1', 'Freshwater acidification', 'damage'),
 ('ImpactWorld+ 2.1', 'Freshwater acidification', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Freshwater eutrophication', 'damage'),
 ('ImpactWorld+ 2.1', 'Freshwater eutrophication', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Land occupation, biodiversity', 'damage'),
 ('ImpactWorld+ 2.1', 'Land occupation, biodiversity', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Land transformation, biodiversity', 'damage'),
 ('ImpactWorld+ 2.1', 'Land transformation, biodiversity', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Marine eutrophication', 'damage'),
 ('ImpactWorld+ 2.1', 'Marine eutrophication', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Terrestrial acidification', 'damage'),
 ('ImpactWorld+ 2.1', 'Terrestrial acidification', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Thermally polluted water', 'damage'),
 ('ImpactWorld+ 2.1', 'Thermally polluted water', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Water availability, freshwater ecosystem', 'damage'),
 ('ImpactWorld+ 2.1', 'Water 

We can loop through the ImpactWorld+ regionalized indicators and print the results.

In [3]:
import matplotlib.pyplot as plt
# method = ('ImpactWorld+ 2.1', 'Land transformation, biodiversity', 'midpoint')
methods = [m for m in get_available_methods() if "ImpactWorld+" in str(m) and "midpoint" in str(m)][:1]
#methods = [('ImpactWorld+ 2.1', 'Land transformation, biodiversity', 'midpoint'),]
for method in methods:
    LCA = EdgeLCIA({act: 1}, method)
    LCA.lci()

    LCA.map_exchanges()
    LCA.map_aggregate_locations()
    LCA.map_dynamic_locations()
    LCA.map_contained_locations()
    LCA.map_remaining_locations_to_global()

    LCA.evaluate_cfs()
    LCA.lcia()
    df = LCA.generate_cf_table()

    if "water" in str(method):
        cutoff = 0.05
        continue
    else:
        cutoff = 0.01

    # remove rows for which "amount" is below the cutoff
    total = df["impact"].sum()
    df = df[df["impact"] > cutoff * total]

    # Plotting
    fig, axes = plt.subplots(3, 2, figsize=(10, 15))

    # 1. Bar plot of "amount" by "supplier name"
    df.groupby("supplier name")["impact"].sum().plot(kind="bar", ax=axes[0, 0])
    axes[0, 0].set_title("Impact by Supplier Name")
    axes[0, 0].set_ylabel("Impact")
    axes[0, 0].tick_params(axis="x", rotation=45)

    # 2. Bar plot of "amount" by "consumer name"
    df.groupby("consumer name")["impact"].sum().plot(kind="bar", ax=axes[0, 1])
    axes[0, 1].set_title("Impact by Consumer Name")
    axes[0, 1].set_ylabel("Impact")
    axes[0, 1].set_xticklabels(
        [label.get_text().replace(", ", "\n") for label in axes[0, 1].get_xticklabels()],
        rotation=45,
        ha="right",
    )

    # 3. Bar plot of "amount" by "consumer location"
    df.groupby("consumer location")["impact"].sum().plot(kind="bar", ax=axes[1, 0])
    axes[1, 0].set_title("Impact by Consumer Location")
    axes[1, 0].set_ylabel("Impact")
    axes[1, 0].tick_params(axis="x", rotation=45)

    # 4. Stacked bar plot for "amount" by "consumer name" and "consumer location"
    df_grouped = df.groupby(["consumer name", "consumer location"])["impact"].sum().unstack()
    df_grouped.plot(kind="bar", stacked=True, ax=axes[1, 1])
    axes[1, 1].set_title("Impact by Consumer Name & Location (Stacked)")
    axes[1, 1].set_ylabel("Impact")
    axes[1, 1].set_xticklabels(
        [label.get_text().replace(", ", "\n") for label in axes[1, 1].get_xticklabels()],
        rotation=45,
        ha="right",
    )
    axes[1, 1].legend(title="Consumer Location", loc="center left", ncol=2)

    # 5. Bar plot of average "CF" by "consumer location"
    df.groupby(["supplier name", "consumer location"])["CF"].mean().plot(kind="bar", ax=axes[2, 0])
    axes[2, 0].set_title("Average CF by Consumer Location")
    axes[2, 0].set_ylabel("CF")
    axes[2, 0].tick_params(axis="x", rotation=45)

    # remove last subplot
    fig.delaxes(axes[2, 1])

    plt.tight_layout()
    plt.show()


/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|██████████| 14940/14940 [00:00<00:00, 112393.37it/s]


Handling static regions...


Processing static groups (pass 1): 100%|██████████| 18/18 [00:00<00:00, 1449.17it/s]


Handling dynamic regions...


Processing dynamic groups (pass 1): 100%|██████████| 2366/2366 [00:10<00:00, 230.56it/s]


Handling contained locations...


Processing contained groups (pass 1): 100%|██████████| 519/519 [00:00<00:00, 21951.73it/s]


Handling remaining exchanges...


In [4]:
LCA.score

0.00030537267708781853

In [4]:
df = LCA.generate_cf_table()

In [11]:
LCA.statistics()

+----------------------+---------------------------------------------+
|       Activity       | market for chlorine, for reuse in zirconium |
|                      |     and hafnium tetrachloride production    |
|     Method name      |       ('ImpactWorld+ 2.1', 'Freshwater      |
|                      |         acidification', 'midpoint')         |
|      Data file       |         ImpactWorld+ 2.1_Freshwater         |
|                      |            acidification_midpoint           |
| Unique CFs in method |                     2418                    |
|   Unique CFs used    |                     356                     |
|  Exc. characterized  |                    10543                    |
| Exc. uncharacterized |                    325366                   |
+----------------------+---------------------------------------------+


In [12]:
df.sort_values("amount", ascending=False)

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
3067,Sulfur dioxide,"(air, urban air close to ground)","electricity production, oil","electricity, high voltage",FR,5.819550e-05,0.564933,3.287658e-05
1540,Nitrogen oxides,"(air, urban air close to ground)","electricity production, oil","electricity, high voltage",FR,4.353023e-05,0.169693,7.386766e-06
3664,Sulfur dioxide,"(air, urban air close to ground)",sulfuric acid production,sulfuric acid,RER,2.748904e-05,1.013358,2.785624e-05
3072,Sulfur dioxide,"(air, non-urban air or from high stacks)","electricity production, hard coal","electricity, high voltage",FR,2.302991e-05,0.564933,1.301037e-05
5120,Sulfur dioxide,"(air,)","sulfur production, petroleum refinery operation",sulfur,RoW,1.746579e-05,0.320052,5.589954e-06
...,...,...,...,...,...,...,...,...
6441,Nitric acid,"(air,)",citric acid production,"protein feed, 100% crude",RoW,1.619446e-48,0.061106,9.895795e-50
1007,Nitric acid,"(air,)",citric acid production,"energy feed, gross",RNA,5.324266e-49,1.170369,6.231355e-49
1008,Nitric acid,"(air,)",citric acid production,"protein feed, 100% crude",RNA,3.713420e-49,1.170369,4.346071e-49
983,Nitric acid,"(air,)",citric acid production,"energy feed, gross",RER,2.440528e-49,0.213900,5.220292e-50


In [6]:
df["CF"].unique()

array([2.14627074e-01, 1.03701824e+00, 1.27154159e-02, 3.62725226e-01,
       1.64280624e-01, 5.11256943e-01, 2.86068890e-01, 2.04003013e-01,
       1.70451775e+01, 3.59239364e-01, 2.09745462e-01, 9.80816828e-02,
       1.37322964e-01, 6.31873982e-01, 8.19714895e-01, 1.57927768e+00,
       1.38567745e-01, 1.30335269e+00, 1.56298950e-01, 3.81978567e+00,
       3.11901496e-01, 4.77011518e-01, 1.60470528e-01, 3.11411337e-01,
       4.27218327e-01, 4.57729915e-02, 3.26688000e-01, 7.38519172e-02,
       1.42424048e-01, 5.19882298e-01, 3.10482782e-01, 1.60522950e-01,
       1.90727939e-01, 8.86117728e-01, 1.12291472e-01, 1.76989269e+00,
       5.12764891e-02, 8.12026897e-01, 5.72559707e+00, 5.12382869e-01,
       2.02801098e-01, 9.98264681e-01, 1.47882836e-01, 4.80848442e-02,
       5.97228678e-01, 2.00683333e-01, 7.02059827e+00, 3.68520407e-01,
       6.84353156e-01, 1.04909844e-01, 5.58194050e+00, 5.08077263e-02,
       1.46084140e-01, 1.88112280e-01, 2.14522571e-02, 3.60626720e-01,
      

In [13]:
df.loc[(df["consumer location"] == "IN-DD")]

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact


In [5]:
df.groupby(["supplier name", "consumer location"])["CF"].mean()

supplier name  consumer location
Ammonia        AR                   0.214627
               AT                   1.037018
               AU                   0.012715
               AU-NSW               0.012715
               AU-QLD               0.012715
                                      ...   
Sulfuric acid  GLO                  0.203823
               RER                  0.661872
               RNA                  3.142301
               RoW                  0.203299
               US                   3.093421
Name: CF, Length: 653, dtype: float64

In [21]:
list(LCA.demand.keys())[0]

124526838126026753

In [6]:
from constructive_geometries import geomatcher
geo = geomatcher.Geomatcher()

geo.contained(
    "CA-QC", biggest_first=False, include_self=False
)

[('ecoinvent', 'Québec, HQ distribution network')]

In [2]:
geo.contained("PG", biggest_first=False)

['PG']

In [6]:
set([x["supplier"]["name"] for x in LCA.ignored_method_exchanges])

{'Water',
 'Water, cooling, unspecified natural origin',
 'Water, in air',
 'Water, lake',
 'Water, river',
 'Water, turbine use, unspecified natural origin',
 'Water, unspecified natural origin',
 'Water, well, in ground'}

In [7]:
LCA.generate_cf_table()

Matrix type: biosphere


,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Water, cooling, unspecified natural origin","(natural resource, in water)","particleboard production, uncoated, from virgi...","particleboard, uncoated",RoW,6.544631e-11,20.179399,1.320667e-09
1,"Water, cooling, unspecified natural origin","(natural resource, in water)","heat and power co-generation, lignite","electricity, high voltage",SI,1.453419e-07,1.087201,1.580158e-07
2,"Water, cooling, unspecified natural origin","(natural resource, in water)",atrazine production,atrazine,RoW,4.228744e-12,19.819034,8.380961e-11
3,"Water, cooling, unspecified natural origin","(natural resource, in water)","sheet rolling, copper","sheet rolling, copper",RoW,1.290820e-10,19.819034,2.558280e-09
4,"Water, cooling, unspecified natural origin","(natural resource, in water)","phosphoric acid production, dihydrate process","phosphoric acid, fertiliser grade, without wat...",MA,5.123237e-08,54.031045,2.768139e-06
...,...,...,...,...,...,...,...,...
4923,"Water, in air","(natural resource, in air)",soy-based resin production,"polyester resin, unsaturated",US,4.729868e-16,9.087076,4.298067e-15
4924,"Water, in air","(natural resource, in air)",soy-based resin production,"polyester resin, unsaturated",RoW,1.378576e-15,20.175187,2.781303e-14
4925,"Water, in air","(natural resource, in air)",clinker production,clinker,RoW,1.625316e-09,20.627229,3.352577e-08
4926,"Water, in air","(natural resource, in air)","particleboard production, cement bonded","particleboard, cement bonded",RER,8.635607e-19,6.750246,5.829247e-18


In [7]:
LCA.ignored_locations

set()

In [9]:
method = ('AWARE 1.2c', 'Country', 'mixed', 'yearly')
convmethod = (
  # 'ecoinvent-3.10',
  'EF v3.1 EN15804',
  'water use',
  'user deprivation potential (deprivation-weighted water consumption)')

In [10]:
import bw2calc

convLCA = bw2calc.LCA({act: 1}, convmethod)
convLCA.lci()
convLCA.lcia()
convLCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


0.06176615560312869